# Pinecone - Primeros pasos

## Introducción <a name="intro"></a>

Lo primero que debemos realizar es crear una **API-Key** sobre la que poder auntenticarnos y, crear un proyecto sobre el que subir nuestros índices.

Por lo tanto, definimos la información necesaria:
+ **PINECONE_API_KEY**
+ **INDEX_NAME**

IMPORTANTE: Para poder trabajar con Pinecone desde Python debemos instalar el paquete que nos da acceso al cliente en la nube https://docs.pinecone.io/docs/python-client

In [1]:
#pip install pinecone-client
#pip install python-dotenv

In [2]:
import os
from pinecone import Pinecone
from dotenv import load_dotenv, find_dotenv
import numpy as np

/Applications/anaconda3/envs/chatbot/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
load_dotenv(find_dotenv()) # busca un .env dentro de los archivos del repo (devuelve True si lo encuentra)

# load_dotenv(path) # si lo tenemos en otra localizacion (o con otro nombre) se puede pasar una ruta

True

In [4]:
PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]

Una vez definidas nuestras constantes para poder interactuar con Pinecone, debemos lanzar el cliente. Esto se consigue a través de la función `Pinecone()` esta función recibe principalmente dos parámetros:
* `api_key`

De esta forma, ya hemos conectado el notebook al servidor de Pinecone

In [5]:
# Importamos Pinecone

pinecone = Pinecone(api_key=PINECONE_API_KEY)

Una vez conectados al servidor, podemos acceder a nuestros índices (creados previamente desde la interfaz de Pinecone) mediante la función `pinecone.list_indexes()` 

In [6]:
pinecone.list_indexes()

{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 384,
              'host': 'chatbot-ska078u.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'chatbot',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}},
             {'deletion_protection': 'disabled',
              'dimension': 1024,
              'host': 'intropinecone-ska078u.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'intropinecone',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

También es posible crear un índice desde cero, todo desde el cliente en Python 

```python
from pinecone import Pinecone

pinecone = Pinecone(api_key=PINECONE_API_KEY)

index_name = "mi_index_prueba"

pinecone.create_index(
        index_name,
        dimension=1536,  
        metric='cosine'
    )

```

Además de poder acceder a su nombre, para controlar el estado del mismo es posible acceder a información como
* Status del índice
* Nombre del índice
* Tamaño del índice

Esto se consigue desde la función `pinecone.describe_index()`

In [7]:
pinecone.describe_index("intropinecone") # Devuelve un índice de valores

{'deletion_protection': 'disabled',
 'dimension': 1024,
 'host': 'intropinecone-ska078u.svc.aped-4627-b74a.pinecone.io',
 'metric': 'cosine',
 'name': 'intropinecone',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'status': {'ready': True, 'state': 'Ready'}}

Como el nombre del índice puede ser algo privado, podemos definirlo en el fichero .env como una variable de entorno

In [8]:
INDEX_NAME = os.environ["INDEX"]

In [9]:
pinecone.describe_index(INDEX_NAME)

{'deletion_protection': 'disabled',
 'dimension': 1024,
 'host': 'intropinecone-ska078u.svc.aped-4627-b74a.pinecone.io',
 'metric': 'cosine',
 'name': 'intropinecone',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'status': {'ready': True, 'state': 'Ready'}}

In [10]:
pinecone.describe_index(INDEX_NAME).dimension

1024

## Trabajando con Índices <a name="index"></a>

Antes de crear vectores propios con archivos PDF (el objetivo central de nuestro asistente conversacional), vamos a crear algunos vectores para subirlos a nuestro índice.

Ahora mismo nuestro índice tiene una dimensionalidad de 1024, vamos a crear algunos vectores aleatorios con Numpy para subir a nuestro índice de Pinecone (serían posibles números de Word Embeddings)

In [11]:
vec_1 = np.random.normal(size = int(pinecone.describe_index(INDEX_NAME).dimension)).tolist() # para subirlos es necesario pasarlos a lista
# vec_1 = np.random.normal(size = 1024).tolist()
vec_2 = np.random.normal(size = int(pinecone.describe_index(INDEX_NAME).dimension)).tolist()

In [12]:
vec_1[:3]

[-1.3724466391637615, -1.0239675170154696, 1.2695066306411473]

Para poder subir estos dos vectores a Pinecone, debemos primero conectarnos al índice, esto se hace desde la función `pinecone.Index()`

In [13]:
index = pinecone.Index(INDEX_NAME)

Una vez que estemos conectados a un índice ya podremos realizar operaciones como:
* __Upsert__ - Insertar vectores
* __Query__  - Búsqueda por similaridad
* __Delete__ - Borrar del índice
* __Update__ - Actualizar el/los elementos del vector


Para subir vectores a un índice emplearemos la función `index.upsert()`. Esta función recibirá los siguientes parámetros principales:
* vectors: Lista de vectores

Esta lista de vectores, comúmente vendrá acompañada de campos que suelen introducirse como diccionario de datos (uno por cada vector) donde se definen los siguientes campos clave:
+ id: nombre del índice
+ values : vector
+ metadata : Valores de metadatos de cada vector (útil para realizar búsqueda semántica)

In [14]:
# Posteriormente haremos esto de forma automatizada con la funcion from_documents

upsert_vectors = index.upsert(  # Insertamos vectores
    vectors = [
        
        {"id"       :"vec_1", 
         "values"   : vec_1,
         "metadata" : {"contenido" : "aprendizaje supervisado"}},
        
        {"id"       :"vec_2", 
         "values"   : vec_2,
         "metadata" : {"contenido" : "aprendizaje NO supervisado"}}        
    ]
)

## Funciones de búsqueda en vectores <a name="query"></a>

Por el momento, nuestras _querys_ serán sin lenguaje natural, ya que aún no hemos subido ningún archivo de texto (nuestros vectores son números que siguen una distribución normal). Cuando trabajemos con lenguaje natural, pasaremos nuestros vectores a embeddings y se hará una búsqueda por similitud.

Para realizar una búsqueda por similaridad en nuestro índice sobre los vectores, podemos hacer uso de la función `index.query()` la cuál recibe otro vector como parámetro y, el parámetro `top_k` que es el número de resultados a desplegar.

In [15]:
query_vector = np.random.normal(size = \
                                int(pinecone.describe_index(INDEX_NAME).dimension)).tolist()

In [16]:
query_vector[:20]

[-0.43275494387056107,
 0.5663124773336229,
 1.490598044016688,
 -2.3820995523017556,
 0.4219687685563165,
 -1.8031085846407877,
 0.8952212535383749,
 1.6509934920883902,
 0.09933257087543773,
 -0.09166285287944981,
 1.0563908010264962,
 0.004441832990409953,
 0.8333596028355444,
 -1.4274799663668332,
 -0.9535794855922187,
 0.7745855714436422,
 -0.28079973498410626,
 1.1697585437701914,
 0.7886382653666958,
 0.6415226356912506]

In [17]:
ask_query = index.query(
    vector = query_vector, # Vector de búsqueda,
    top_k  = 1 # Solo el resultado con la similaridad más alta
)

El resultado almacenado en esta variable será simplemente los id de los vectores con la similariadad más alta, tantos como le hayamos pasado al parámetro `top_k`

In [18]:
ask_query

{'matches': [], 'namespace': '', 'usage': {'read_units': 1}}

In [19]:
index.query(
    vector = query_vector, # Vector de búsqueda,
    top_k  = 2 # dos resultados con similitud más alta
)

{'matches': [], 'namespace': '', 'usage': {'read_units': 1}}

También es posible hacer una busqueda semántica mediante metadatos. https://docs.pinecone.io/docs/metadata-filtering.

La búsqueda semántica te va a permitir filtrar para que solo busque en vectores que cumplan ciertas condiciones.

In [20]:
# Incluimos mas vectores de prueba

index.upsert([
    ("A", np.random.normal(size = int(pinecone.describe_index(INDEX_NAME).dimension)).tolist(), 
     {"genre": "comedy", "year": 2020}),
    
    ("B", np.random.normal(size = int(pinecone.describe_index(INDEX_NAME).dimension)).tolist(),
     {"genre": "documentary", "year": 2019}),
    
    ("C", np.random.normal(size = int(pinecone.describe_index(INDEX_NAME).dimension)).tolist(),
     {"genre": "comedy", "year": 2019}),
    
    ("D", np.random.normal(size = int(pinecone.describe_index(INDEX_NAME).dimension)).tolist(), 
     {"genre": "drama"}),
    
    ("E", np.random.normal(size = int(pinecone.describe_index(INDEX_NAME).dimension)).tolist(), 
     {"genre": "drama"})
])

{'upserted_count': 5}

In [21]:
query_vector = np.random.normal(size = int(pinecone.describe_index(INDEX_NAME).dimension)).tolist()

In [22]:
index.query(
    vector=query_vector,
    filter={
        "genre": {"$eq": "comedy"}, #$ne (!=), $gt (>), $gte (>=)...
        "year": 2019
    },
    top_k=1,
    include_metadata=True # para que filtre por metadatos hay que añadir este parametro
)

{'matches': [], 'namespace': '', 'usage': {'read_units': 1}}

Para borrar vectores, se puede emplear la función `index.delete()` Es posible eliminar directamente IDs concretos sobre el campo `ids` o borrar por metadatos sobre el campo `filter`

In [23]:
index.delete(ids=["A", "B"])

{}

También es posible borrar vectores mediante el filtrado de metadatos, no obstante esta operación no está habilitada para las cuentas que tengan un entorno de computación starter

```python
index.delete(
    filter={
        "genre": {"$eq": "documentary"},
        "year": 2019
    }
)
```

Para borrar todos los vectores de un índice podemos hacer uso del parámetro `delete_all`, no obstante, tampoco es posible realizar esta operación en cuentas tipo Starter.

```python
# borrar todos los vectores
index.delete(delete_all=True)
```

Para actualizar un vector se puede realizar desde la función `index.update()` https://docs.pinecone.io/docs/manage-data#updating-records

In [24]:
# Creamos un vector para comprobar la operación update

vec_test = np.random.normal(size = int(pinecone.describe_index(INDEX_NAME).dimension)).tolist()

upsert_vectors = index.upsert(  # Insertamos vectores
    vectors = [
        
        {"id"       :"test", 
         "values"   : vec_test,
         "metadata" : {"contenido" : "aprendizaje supervisado"}},    
    ]
)

In [25]:
# Actualizamos el vector
index.update(id="test", 
             values= np.ones(int(pinecone.describe_index(INDEX_NAME).dimension)).tolist(), 
             set_metadata={"contenido": ["aprendizaje supervisado", "Estadística"],
                           "fecha" : "11/10/2024"}
                           )

{}